In [14]:
!pip install termcolor


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# import urllib.request
# import zipfile
# import os

# # URL of the MovieLens dataset
# url = "https://files.grouplens.org/datasets/movielens/ml-latest-small.zip"
# zip_file = "ml-latest-small.zip"
# extract_folder = "ml-latest-small"

# # Download the zip file
# print("Downloading dataset...")
# urllib.request.urlretrieve(url, zip_file)
# print("Download completed!")

# # Create extraction folder if it doesn't exist
# os.makedirs(extract_folder, exist_ok=True)

# # Unzip the file
# print("Extracting dataset...")
# with zipfile.ZipFile(zip_file, 'r') as zip_ref:
#     zip_ref.extractall(extract_folder)
# print("Extraction completed!")

# # Check files
# print("Files extracted:", os.listdir(extract_folder))


Download completed!
Extracting dataset...
Extraction completed!
Files extracted: ['ml-latest-small']


In [2]:
import pandas as pd
import os

# Folder where the dataset was extracted
dataset_folder = "ml-latest-small"

# Load CSV files
movies_df = pd.read_csv(os.path.join(dataset_folder, "movies.csv"))
ratings_df = pd.read_csv(os.path.join(dataset_folder, "ratings.csv"))
tags_df = pd.read_csv(os.path.join(dataset_folder, "tags.csv"))
links_df = pd.read_csv(os.path.join(dataset_folder, "links.csv"))

movies_links_df = pd.merge(movies_df, links_df, on="movieId", how="left")
movies_ratings_df = pd.merge(ratings_df, movies_links_df, on="movieId", how="left")
final_df = pd.merge(movies_ratings_df, tags_df, on="movieId", how="left")
final_df['tag'] = final_df['tag'].fillna('')
final_df.drop(columns=['userId_y'],axis=1, inplace=True)
final_df.rename(columns={'userId_x' : 'userId',
                         'timestamp_x' : 'timestamp',
                         'timestamp_y' : 'time_going'}, inplace=True)

print(final_df.head())
final_df.to_csv(os.path.join("final_combined.csv"), index=False)
print("final_combined.csv")


   userId  movieId  rating  timestamp                    title  \
0       1        1     4.0  964982703         Toy Story (1995)   
1       1        1     4.0  964982703         Toy Story (1995)   
2       1        1     4.0  964982703         Toy Story (1995)   
3       1        3     4.0  964981247  Grumpier Old Men (1995)   
4       1        3     4.0  964981247  Grumpier Old Men (1995)   

                                        genres  imdbId   tmdbId    tag  \
0  Adventure|Animation|Children|Comedy|Fantasy  114709    862.0  pixar   
1  Adventure|Animation|Children|Comedy|Fantasy  114709    862.0  pixar   
2  Adventure|Animation|Children|Comedy|Fantasy  114709    862.0    fun   
3                               Comedy|Romance  113228  15602.0  moldy   
4                               Comedy|Romance  113228  15602.0    old   

     time_going  
0  1.139046e+09  
1  1.137207e+09  
2  1.525286e+09  
3  1.143425e+09  
4  1.143425e+09  
final_combined.csv


In [3]:
data = pd.read_csv('final_combined.csv')
df = data.copy()
df.head()

,userId,movieId,rating,timestamp,title,genres,imdbId,tmdbId,tag,time_going
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,pixar,1.139046e+09
1,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,pixar,1.137207e+09
2,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,fun,1.525286e+09
3,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0,moldy,1.143425e+09
4,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0,old,1.143425e+09


In [4]:
print(df.isna().sum()/(df.shape[0]))  # missing null values in persentage
# drop all null values as we have 18% missing values in total, so wont affect the data.
df.dropna(inplace=True)

userId        0.000000
movieId       0.000000
rating        0.000000
timestamp     0.000000
title         0.000000
genres        0.000000
imdbId        0.000000
tmdbId        0.000045
tag           0.183891
time_going    0.183891
dtype: float64


In [5]:
df0 = df.groupby(['userId', 'movieId']).agg(
    title=('title', 'first'),           # Keep the movie title
    genres=('genres', 'first'),         # Keep genres
    timestamp=('timestamp', 'first'),   # keep timestamp
    imdbId=('imdbId', 'first'),         # Keep IMDb ID
    tmdbId=('tmdbId', 'first'),         # Keep TMDB ID
    tags=('tag', lambda x: '|'.join(set(x)) if x.notna().any() else ''),  # Combine all tags
    time_start=('time_going', 'min'),   # Earliest timestamp
    time_end=('time_going', 'max')      # Latest timestamp
).reset_index()

df0['total_time_taken'] = round((df0['time_end'] - df0['time_start'])/(60*60*60*60*24), 2)

df0.head()

,userId,movieId,title,genres,timestamp,imdbId,tmdbId,tags,time_start,time_end,total_time_taken
0,1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,964982703,114709,862.0,fun|pixar,1.137207e+09,1.525286e+09,1.25
1,1,3,Grumpier Old Men (1995),Comedy|Romance,964981247,113228,15602.0,moldy|old,1.143425e+09,1.143425e+09,0.00
2,1,47,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,964983815,114369,807.0,mystery|twist ending|serial killer,1.137206e+09,1.457842e+09,1.03
3,1,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,964982931,114814,629.0,thriller|tricky|suspense|mindfuck|twist ending...,1.137207e+09,1.457842e+09,1.03
4,1,101,Bottle Rocket (1996),Adventure|Comedy|Crime|Romance,964980868,115734,13685.0,off-beat comedy|quirky|crime,1.138137e+09,1.525287e+09,1.24


In [6]:
# for converting to a matrix format we need just userId x moviesId alone and movies will be marked 1,2,3, 
# based on the order of timestamp and where movies that are selected like total_time_taken > 35% of max time.

df1 = df0.sort_values(by=['userId', 'timestamp'], ascending=True)
df1['watch_order'] = df1.groupby('userId').cumcount() + 1
df2 = df1.pivot(index='userId', columns='movieId', values='watch_order')
df2 = df2.fillna(0).astype(int)
df2.to_csv('user_movie_matrix.csv', index=False)
print(df2)


movieId  1       2       3       5       7       11      14      16      \
userId                                                                    
1            67       0      18       0       0       0       0       0   
2             0       0       0       0       0       0       0       0   
3             0       0       0       0       0       0       0       0   
4             0       0       0       0       0       0       0       0   
5            19       0       0       0       0       0       0       0   
...         ...     ...     ...     ...     ...     ...     ...     ...   
606         444       0       0       0     231     356       0       0   
607          83       0       0       0       0      96       0       0   
608          96     182     248       0       0       0       0     340   
609          17       0       0       0       0       0       0       0   
610          43       0       0       0       0       0       0      15   

movieId  17      21     

In [7]:
# Each user's watched movies in order

movie_ids = df2.columns.tolist()
movie2idx = {movie: idx+1 for idx, movie in enumerate(movie_ids)}
idx2movie = {idx+1: movie for idx, movie in enumerate(movie_ids)}
num_movies = len(movie_ids) + 1

user_sequences = []
for idx, row in df2.iterrows():
    seq = [movie2idx[col] for col, order in row.items() if order > 0]
    if len(seq) > 1:
        user_sequences.append(seq)

print(f'the length of the user where we have the patten will be predicted {len(user_sequences)}')

the length of the user where we have the patten will be predicted 610


In [8]:
# Find the length of each sequence
sequence_lengths = [len(seq) for seq in user_sequences]
max_seq_len = max(sequence_lengths)

print(f"The maximum sequence length is: {max_seq_len}, with this we will be defiing the seq_len for the model")

The maximum sequence length is: 1198, with this we will be defiing the seq_len for the model


In [114]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity

train_seqs, test_seqs = train_test_split(user_sequences, test_size=0.2, random_state=42)

# Dataset for RNN predicting next movie
class MovieSequenceDataset(Dataset):
    def __init__(self, sequences, seq_len=30):
        self.sequences = sequences
        self.seq_len = seq_len

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seq = self.sequences[idx]
        if len(seq) <= 1:
            return None
        # Input: first seq_len movies
        x = seq[:self.seq_len]
        # Target: next movie
        y = seq[len(x)] if len(seq) > len(x) else seq[-1]
        if len(x) < self.seq_len:
            x = [0]*(self.seq_len - len(x)) + x
        return torch.tensor(x, dtype=torch.long), torch.tensor(y, dtype=torch.long)

def collate_fn(batch):
    batch = [b for b in batch if b is not None]  # filter empty
    xs, ys = zip(*batch)
    xs = torch.stack(xs)
    ys = torch.stack(ys)
    return xs, ys

train_dataset = MovieSequenceDataset(train_seqs)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

test_dataset = MovieSequenceDataset(test_seqs)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

# RNN Model with more hidden layers
num_movies = df2.shape[1] + 1  # +1 for padding

class RNNRecommender(nn.Module):
    def __init__(self, num_movies, embedding_dim=128, hidden_dim=256, gru_layers=2, fc_hidden=128):
        super().__init__()
        self.embedding = nn.Embedding(num_movies, embedding_dim, padding_idx=0)
        self.rnn = nn.GRU(
            embedding_dim, 
            hidden_dim, 
            num_layers=gru_layers, 
            batch_first=True, 
        )
        # Fully connected layers after GRU
        self.fc1 = nn.Linear(hidden_dim, fc_hidden)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(fc_hidden, num_movies)  # output logits

    def forward(self, x):
        x = self.embedding(x)         # [batch_size, seq_len, embedding_dim]
        out, h = self.rnn(x)          # out: [batch_size, seq_len, hidden_dim], h: [num_layers, batch, hidden_dim]
        last_hidden = h[-1]           # take last layer's hidden state
        x = self.fc1(last_hidden)
        x = self.relu(x)
        logits = self.fc2(x)
        return logits                 # [batch_size, num_movies]

# Instantiate model
model = RNNRecommender(num_movies, embedding_dim=128, hidden_dim=256, gru_layers=2, fc_hidden=128)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Training
for epoch in range(180):
    model.train()
    total_loss = 0
    for x, y in train_loader:
        optimizer.zero_grad()
        logits = model(x)        # [batch_size, num_movies]
        loss = criterion(logits, y)  # y = next movie index
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} Loss: {total_loss/len(train_loader):.4f}")





Epoch 1 Loss: 7.2689
Epoch 2 Loss: 6.1962
Epoch 3 Loss: 5.2575
Epoch 4 Loss: 4.6729
Epoch 5 Loss: 4.1430
Epoch 6 Loss: 3.5106
Epoch 7 Loss: 2.8883
Epoch 8 Loss: 2.2784
Epoch 9 Loss: 1.6562
Epoch 10 Loss: 1.1296
Epoch 11 Loss: 0.7118
Epoch 12 Loss: 0.4431
Epoch 13 Loss: 0.2724
Epoch 14 Loss: 0.1770
Epoch 15 Loss: 0.1233
Epoch 16 Loss: 0.0917
Epoch 17 Loss: 0.0736
Epoch 18 Loss: 0.0574
Epoch 19 Loss: 0.0433
Epoch 20 Loss: 0.0360
Epoch 21 Loss: 0.0300
Epoch 22 Loss: 0.0252
Epoch 23 Loss: 0.0217
Epoch 24 Loss: 0.0190
Epoch 25 Loss: 0.0166
Epoch 26 Loss: 0.0147
Epoch 27 Loss: 0.0132
Epoch 28 Loss: 0.0121
Epoch 29 Loss: 0.0113
Epoch 30 Loss: 0.0101
Epoch 31 Loss: 0.0094
Epoch 32 Loss: 0.0087
Epoch 33 Loss: 0.0081
Epoch 34 Loss: 0.0074
Epoch 35 Loss: 0.0070
Epoch 36 Loss: 0.0064
Epoch 37 Loss: 0.0061
Epoch 38 Loss: 0.0058
Epoch 39 Loss: 0.0054
Epoch 40 Loss: 0.0051
Epoch 41 Loss: 0.0048
Epoch 42 Loss: 0.0044
Epoch 43 Loss: 0.0043
Epoch 44 Loss: 0.0040
Epoch 45 Loss: 0.0039
Epoch 46 Loss: 0.00

In [115]:
torch.save(model.state_dict(), 'model_ep180_hd256_fc128_lr001.pth')

In [ ]:
model.load_state_dict(torch.load('model_256_0_001.pth'))  

# model_256_0_001.pth -- 0.377   ---- Best model 
# model_ep150_hd256_fc128_lr001 -- 0.354 
# model_ep180_hd256_fc128_lr001 -- 0.328
# model_ep180_hd2-256_hd1-256_fc128_lr001.pth -- 0.311
# model_ep180_hd512_fc128_lr001.pth -- 0.066
# model_ep180_hd256_fc256_lr001.pth -- 0.057


C:\Users\gk149\AppData\Local\Temp\ipykernel_5596\3974688159.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_256_0_001.pth'))


<All keys matched successfully>

In [121]:
from termcolor import colored
from tabulate import tabulate

top_k = 20
model.eval()
total = 0
correct_topk = 0

results = []

for x, y_seq in test_loader:
    with torch.no_grad():
        logits = model(x)
        probs = torch.softmax(logits, dim=1).numpy()

        for i in range(x.size(0)):
            # Top-K predicted movies for this sample
            topk_preds = probs[i].argsort()[-top_k:][::-1]  # reverse for descending
            true_movies = y_seq[i].numpy()
            true_movies = true_movies[true_movies > 0]

            if len(true_movies) == 0:
                continue

            num_matches = len(set(topk_preds) & set(true_movies))
            match_ratio = num_matches / len(true_movies)

            # ✅ Hit if at least 50% of true movies are in Top-K
            status = "✅ HIT" if match_ratio >= 0.5 else "❌ MISS"

            # Color formatting
            colored_preds = [
                colored(str(m), "green") if m in true_movies else str(m)
                for m in topk_preds[:10]
            ]
            colored_true = [colored(str(m), "cyan", attrs=["bold"]) for m in true_movies]

            results.append([
                total + 1,
                ", ".join(colored_true),
                ", ".join(colored_preds),
                f"{num_matches}/{len(true_movies)}",
                status
            ])

            if match_ratio >= 0.5:
                correct_topk += 1

            total += 1

# Final accuracy
topk_accuracy = correct_topk / total


In [122]:
output_file = "evaluation_summary.txt"
headers = ["#", "True Movies", "Top-K Predictions", "Matches", "Status"]
table_str = tabulate(results, headers=headers, tablefmt="fancy_grid", stralign="center")
summary_text = f"\nTop-{top_k} Recommendation Accuracy (≥50% match rule): {topk_accuracy:.3f}\n"
print(table_str)
print(colored(summary_text, "yellow", attrs=["bold"]))

with open(output_file, "w", encoding="utf-8") as f:
    f.write(table_str + "\n")
    f.write(summary_text)

print(colored(f"\n✅ Results saved to: {output_file}", "green", attrs=["bold"]))


╒═════╤═══════════════╤══════════════════════════════════════════════════════════╤═══════════╤══════════╕
│   # │   True Movies │                    Top-K Predictions                     │  Matches  │  Status  │
╞═════╪═══════════════╪══════════════════════════════════════════════════════════╪═══════════╪══════════╡
│   1 │           288 │    306, 331, 389, 288, 934, 263, 257, 293, 1416, 290     │    1/1    │  ✅ HIT  │
├─────┼───────────────┼──────────────────────────────────────────────────────────┼───────────┼──────────┤
│   2 │           147 │     146, 149, 147, 157, 687, 393, 578, 145, 367, 133     │    1/1    │  ✅ HIT  │
├─────┼───────────────┼──────────────────────────────────────────────────────────┼───────────┼──────────┤
│   3 │           166 │    147, 152, 157, 171, 160, 149, 172, 167, 1234, 972     │    0/1    │ ❌ MISS  │
├─────┼───────────────┼──────────────────────────────────────────────────────────┼───────────┼──────────┤
│   4 │            51 │      49, 939, 77, 183, 83